## Validation of tension field theory relaxed energy densities

In [ ]:
import energy, tensors
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt
import fd_validation

et = tensors.ElasticityTensor2D()
et.setIsotropic(1, 0.3)
et.setOrthotropic(1, 1, 0.3, 1)

def getPerturbedC(eps):
    F = np.identity(2) + eps * np.random.uniform(size=(2, 2))
    return F.T @ F

psi_C = energy.StVenantKirchhoffCBased(et)
n = np.random.uniform(size=2)
n /= np.linalg.norm(n)

In [ ]:
iwsp = energy.IsotropicWrinkleStrainProblem(psi_C, getPerturbedC(1e-3), n)

plt.figure(figsize=(8,4))
plt.subplot(1, 2, 1)
fd_validation.gradConvergencePlot(iwsp)
plt.subplot(1, 2, 2)
fd_validation.hessConvergencePlot(iwsp)
plt.tight_layout()

In [ ]:
awsp = energy.AnisotropicWrinkleStrainProblem(psi_C, getPerturbedC(1e-3), n)

plt.figure(figsize=(8,4))
plt.subplot(1, 2, 1)
fd_validation.gradConvergencePlot(awsp)
plt.subplot(1, 2, 2)
fd_validation.hessConvergencePlot(awsp)
plt.tight_layout()

In [ ]:
def getRelaxedStVk(dim):
    return energy.RelaxedStVenantKirchhoffMembrane(et)

In [ ]:
e = energy.RelaxedStVenantKirchhoffMembrane(et)
F = e.getDeformationGradient()
F[0, 0] += 0.01
F[1, 1] += 0.01
e.setDeformationGradient(F)
e.tensionState()

In [ ]:
def getPerturb(F):
    return np.random.uniform(-1, 1, F.shape)

def fd_approx(e, f, F, dF, eps = 1e-6):
    e.setDeformationGradient(F + eps * dF)
    plus = f()
    e.setDeformationGradient(F - eps * dF)
    minus = f()
    e.setDeformationGradient(F)
    return (plus - minus) / (2 * eps)

def relerror(fd, an):
    den = norm(an)
    if (den == 0.0): den = 1
    return norm(fd - an) / den

def denergy_validation_raw(e, F, dF, eps = 1e-6):
    fd = fd_approx(e, lambda: e.energy(), F, dF, eps)
    an = e.denergy(dF)
    return fd, an
def denergy_validation(e, F, dF, eps = 1e-6):
    return relerror(*denergy_validation_raw(e, F, dF, eps))

def delta_denergy_validation_raw(e, F, dF, eps = 1e-6):
    fd = fd_approx(e, lambda: e.denergy(), F, dF, eps)
    an = e.delta_denergy(dF)
    return fd, an
def delta_denergy_validation(e, F, dF, eps = 1e-6):
    return relerror(*delta_denergy_validation_raw(e, F, dF, eps))

def delta2_denergy_validation_raw(e, F, dF_a, dF_b, eps = 1e-6):
    fd = fd_approx(e, lambda: e.delta_denergy(dF_a), F, dF_b, eps)
    an = e.delta2_denergy(dF_b, dF_a)
    return fd, an
def delta2_denergy_validation(e, F, dF_a, dF_b, eps = 1e-6):
    return relerror(*delta2_denergy_validation_raw(e, F, dF_a, dF_b, eps))

def genPlots(getEnergy):
    def test(dim):
        e = getEnergy(dim)
        F = e.getDeformationGradient()
        minDim = min(F.shape)
        F[0:minDim, 0:minDim] = np.identity(minDim)
        F = e.getDeformationGradient()
        F[0, 0] += 0.05
        F[1, 1] -= 0.09
        #F += 1e-2 * getPerturb(F)
        #F = np.array([[ 1.10053726e+00,  3.90594218e-04], 
        #      [-2.18652537e-03,  1.00972438e+00], 
        #      [-5.26725946e-03,  2.06696459e-03]])
        print(F)
        e.setDeformationGradient(F)
        print(e.tensionState())

        dF_a = getPerturb(F)
        dF_b = getPerturb(F)

        epsilons = np.logspace(-2, -8, 100)
        plt.loglog(epsilons, [       denergy_validation(e, F, dF_a,       eps) for eps in epsilons], label=       'denergy')
        plt.loglog(epsilons, [ delta_denergy_validation(e, F, dF_a,       eps) for eps in epsilons], label='delta  denergy')
        try: # Some energies do not implement third derivatives
            plt.loglog(epsilons, [delta2_denergy_validation(e, F, dF_a, dF_b, eps) for eps in epsilons], label='delta2 denergy')
        except: pass
        plt.legend()
        name = e.__class__.__name__
        try: name += " (Isotropic)" if e.isIsotropic() else "  (Orthotropic)"
        except: pass
        plt.title(name)
        plt.grid()
    fig = plt.figure(figsize=(10, 4))
    for dim in [2, 3]:
        plt.subplot(1, 2, dim - 1)
        test(dim)
    plt.tight_layout()

In [ ]:
for ge in [getRelaxedStVk]:
    genPlots(ge)

In [ ]:
# Force-strain angular plot

In [ ]:
dx = 0.001
F = np.array([[1 + dx, 0], [0, 1 - dx], [0, 0]]) #Uniaxial stretch
R = lambda t: np.array([[np.cos(t), -np.sin(t)], [np.sin(t), np.cos(t)]])

In [ ]:
#et.setOrthotropic(1, 1, 0.3, 0.3)
mu = 17 / 2
nu = 1 - 1 / (2 * mu) # Set the Poisson's ratio so that uniaxial tension in the 45 degree direction produces a pure shear state
#nu = 0.48
#nu = 0
et.setOrthotropic(1, 1, nu, mu)
psi_C.elasticityTensor = et
e.psi().elasticityTensor = et
Frot = F @ R(th).T

In [ ]:
# We need a surprisingly high Poisson's ratio to get this extreme a stiffness variation.
nu

In [ ]:
# This high Poisson's ratio actually makes sense if we consider the Homogenized properties of
# a + shaped tiling...
et.setOrthotropic(0.45691, 0.456844, 0.978457, 9.70376)
psi_C.elasticityTensor = et
e.psi().elasticityTensor = et

In [ ]:
import plots, importlib
importlib.reload(plots)
plots.tensionForcePolarPlot(0.001, [e, psi_C], ['$\psi_r$', '$\psi$'], [True, False])